In [1]:
#import json

#path_file="/notebook/nas-trainings/arne/DGFISMA/reporting_obligations/code/DGFISMA_reporting_obligations/input_dir/doc_27540107-b189-54b8-83a7-d20c6524b424.jsonl"
#jsonl=open( path_file ).read().rstrip("\n").split( "\n" )
#json_data=json.loads( jsonl[0])

In [1]:
#with open( "/notebook/nas-trainings/arne/DGFISMA/reporting_obligations/code/DGFISMA_reporting_obligations/input_dir/doc_27540107-b189-54b8-83a7-d20c6524b424.html" , "w"  ) as f:
#    f.write( json_data['content_html'][0]  )

In [45]:
#Test code Francois, processing of lists 
import os
import re
from spacy.lang.en import English # updated

import sys
sys.path.append( "/notebook/nas-trainings/arne/DGFISMA/reporting_obligations/code/DGFISMA_reporting_obligations/" )

sentencizer = English()
sentencizer.add_pipe(sentencizer.create_pipe('sentencizer'))

INPUT_DIR="/notebook/nas-trainings/arne/DGFISMA/reporting_obligations/code/DGFISMA_reporting_obligations/tests/test_files/txt/"

FILENAME="small_nested_tables.txt"
#FILENAME="doc_bf4ef384-bd7a-51c8-8f7d-d2f61865d767.txt"

input_sentences = open( os.path.join( INPUT_DIR, FILENAME ) )  #.read() #.strip( "\n" ).split( "\n" )


In [46]:
class seekable_iterator:
    def __init__(self, iterator):
        self.iterator = iterator
        self.current = None
        self.reuse = False

    def __iter__(self):
        return self
    
    def __next__(self):
        return self.next()

    def next(self):
        if self.reuse:
            self.reuse = False
        else:
            self.current = None
            self.current = next(self.iterator)
        return self.current

    def rewind(self):
        self.reuse = True

In [47]:
input_sentences = seekable_iterator(iter(input_sentences))

for main_line in input_sentences:
    main_line = main_line.rstrip("\n")
    #detect sentences like "Of the following:", but not "Of the following: some more". I.e. we detect the start of an enumeration.
    if re.search(r'[:]\s*$',main_line):
        #split paragraph in sentences (using spacy sentencizer), and 
        main_sentences = list(map(lambda s: re.sub(r'[)](?![ ,;:.?! ()])(?!th)', r') ',s.text.rstrip()), sentencizer(main_line).sents))
        last_sentence = main_sentences.pop()
        print( last_sentence )
    
    #print( main_line ) 

  The total amount to be raised in accordance with Article 3 shall be allocated as follows:
(a) 95 % to the following entities and groups:
  For the purpose of applying the provisions referred to in paragraph 2:


In [48]:
input_sentences = open( os.path.join( INPUT_DIR, FILENAME ) )  #.read() #.strip( "\n" ).split( "\n" )
input_sentences = seekable_iterator(iter(input_sentences))
input_sentences.next()

'Article 3\n'

In [59]:
input_sentences.next()

'1.   The total amount to be raised in accordance with Article 3 shall be allocated as follows:\n'

In [60]:
from process_article_lists import collect_sub_lines

other_lines=collect_sub_lines( input_sentences, separator=None )
other_lines

['95 % to the following entities and groups ❬entities and groups referred to in Article 7(2) of Regulation (EU) No 806/2014 ‖ and/or entities and groups with regard to which the Board has decided to exercise powers under Regulation (EU) No 806/2014 pursuant to Article 7(4)(b) of that Regulation ‖ and/or entities and groups in relation to which participating Member States have decided in accordance with Article 7(5) of Regulation (EU) No 806/2014 that the Board exercise powers and responsibilities conferred to it by that Regulation❭',
 '5 % to entities and groups referred to in Article 7(3) of Regulation (EU) No 806/2014']

### END

In [48]:
print( "originator means an entity which ❬itself or through related entities, directly or indirectly, was involved in the original agreement which created the obligations or potential obligations of the debtor or potential debtor giving rise to the exposure being securitised ‖ and/or purchases a third party\'s exposures for its own account and then securitises them❭")

originator means an entity which ❬itself or through related entities, directly or indirectly, was involved in the original agreement which created the obligations or potential obligations of the debtor or potential debtor giving rise to the exposure being securitised ‖ and/or purchases a third party's exposures for its own account and then securitises them❭


In [49]:
for line in other_lines:
    if 'recognised third-country investment' in line:
        print( line )

"recognised third-country investment firm" means a firm meeting all of the following conditions ❬if it were established within the Union, it would be covered by the definition of an investment firm ‖ and/or it is authorised in a third country ‖ and/or it is subject to and complies with prudential rules considered by the competent authorities as at least as stringent as those laid down in this Regulation or in Directive 2013/36/EU❭


In [126]:
import re
sentence="test something sdf :"
re.search(r'[:]\s*$', sentence  )  #this regex finds all :'s'

<re.Match object; span=(19, 20), match=':'>

In [105]:
#read in cassis 
#read in the json:
import json
import base64

from cassis.typesystem import load_typesystem
from cassis.xmi import load_cas_from_xmi


with open( "/notebook/nas-trainings/arne/DGFISMA_paragraph_detection/tests/test_files/typesystems/typesystem.xml"   , 'rb') as f:
    typesystem = load_typesystem(f)

with open('/notebook/nas-trainings/arne/DGFISMA_paragraph_detection/tests/test_files/json/doc_bf4ef384-bd7a-51c8-8f7d-d2f61865d767.json') as json_file:
    response = json.load(json_file)

decoded_cas_response=base64.b64decode( response[ 'cas_content' ] ).decode( 'utf-8' )

cas_response=load_cas_from_xmi( decoded_cas_response, typesystem=typesystem  )

#all_paragraphs=list(cas_response.get_view( 'html2textView' ).select( "de.tudarmstadt.ukp.dkpro.core.api.segmentation.type.Paragraph" ) )


In [107]:
with open( "/notebook/nas-trainings/arne/DGFISMA/reporting_obligations/code/DGFISMA_reporting_obligations/tests/test_files/txt/doc_bf4ef384-bd7a-51c8-8f7d-d2f61865d767.txt" , "w"  ) as f: 
    for tag in list( cas_response.get_view( 'html2textView' ).select( "com.crosslang.uimahtmltotext.uima.type.ValueBetweenTagType"  ) ):
        if tag.tagName == 'p':
            f.write( f"{tag.get_covered_text()}\n" )

In [84]:
list( cas_response.get_view( 'html2textView' ).select( "com.crosslang.uimahtmltotext.uima.type.ValueBetweenTagType"  ) )[14].get_covered_text()

'95 % to the following entities and groups:'

In [90]:
list( cas_response.get_view( 'html2textView' ).select( "com.crosslang.uimahtmltotext.uima.type.ValueBetweenTagType"  ) )[7]

com_crosslang_uimahtmltotext_uima_type_ValueBetweenTagType(xmiID=3083, text=None, tagName='p', attributes="class='normal'", begin=702, end=796, type='com.crosslang.uimahtmltotext.uima.type.ValueBetweenTagType')

In [93]:
paragraph=list(cas_response.get_view( 'html2textView' ).select( "de.tudarmstadt.ukp.dkpro.core.api.segmentation.type.Paragraph" ) )[0]

list( cas_response.get_view( 'html2textView' ).select_covered("com.crosslang.uimahtmltotext.uima.type.ValueBetweenTagType" , paragraph ) )

[com_crosslang_uimahtmltotext_uima_type_ValueBetweenTagType(xmiID=3083, text=None, tagName='p', attributes="class='normal'", begin=702, end=796, type='com.crosslang.uimahtmltotext.uima.type.ValueBetweenTagType'),
 com_crosslang_uimahtmltotext_uima_type_ValueBetweenTagType(xmiID=3090, text=None, tagName='table', attributes="width='100%'border='0'cellspacing='0'cellpadding='0'", begin=801, end=2447, type='com.crosslang.uimahtmltotext.uima.type.ValueBetweenTagType'),
 com_crosslang_uimahtmltotext_uima_type_ValueBetweenTagType(xmiID=3097, text=None, tagName='tbody', attributes='', begin=825, end=2442, type='com.crosslang.uimahtmltotext.uima.type.ValueBetweenTagType'),
 com_crosslang_uimahtmltotext_uima_type_ValueBetweenTagType(xmiID=3104, text=None, tagName='tr', attributes='', begin=836, end=2434, type='com.crosslang.uimahtmltotext.uima.type.ValueBetweenTagType'),
 com_crosslang_uimahtmltotext_uima_type_ValueBetweenTagType(xmiID=3111, text=None, tagName='td', attributes="valign='top'", be

In [ ]:
for sentence in cas.select('cassis.Sentence'):
    for token in cas.select_covered('cassis.Token', sentence):
        print(token.get_covered_text())

In [16]:
from spacy.lang.en import English # updated

sentencizer = English()
sentencizer.add_pipe(sentencizer.create_pipe('sentencizer'))
sentencizer(main_line).sents

In [17]:
main_line

'The deadlines in the first and second subparagraphs of this paragraph may be extended by an additional six months where the Commission has adopted the implementing act referred to in Article 497(3) of Regulation (EU) No 575/2013."'

In [19]:
list( sentencizer( "Then happens something. Some more" ).sents )

[Then happens something., Some more]

In [127]:
#this main_line is the first p of a paragraph...
main_line="the purposes of this Regulation, the following 4545)th df,"
main_sentences = list(map(lambda s: re.sub(r'[)](?![ ,;:.?! ()])(?!th)', r') ',s.text.rstrip()), sentencizer(main_line).sents ))

In [128]:
main_sentences

['the purposes of this Regulation, the following 4545)th df,']

In [111]:
main_line="the purposes of this Regulation, the following 4545)th,"
list( sentencizer(main_line).sents )[0]
re.sub(r'[)](?![ ,;:.?! ()])(?!th)', r') ' , list( sentencizer(main_line).sents )[0]  ) 


TypeError: expected string or bytes-like object

In [116]:
list( sentencizer(main_line).sents )[0].text.rstrip()

'the purposes of this Regulation, the following 4545)th,'

In [99]:
main_line="the purposes of this Regulation, the following 4545)df,"
list( sentencizer(main_line).sents )

[the purposes of this Regulation, the following 4545)df,]

In [50]:
input_sentence=' 9. f'
re.sub(r'^ *[0-9]*[.] *', r'', input_sentence)

'f'

In [122]:
main_line="the purposes of this Regulation, the following 4545)th "
re.sub(r'[)](?![ ,;:.?! ()])(?!th)', r') ' , main_line  ) 

'the purposes of this Regulation, the following 4545)th '

In [89]:
(?!foo)

re.sub(  )

#Negative Lookahead	Asserts that what immediately follows the current position in the string is not foo

SyntaxError: invalid syntax (<ipython-input-89-07c269a6781d>, line 1)